# 此样例是二分类问题，建立带有一个包含四个神经元的隐藏层的神经网络
## 输入
x1, x2
## 输出
y = x1 + x2 >= 0? 1: 0;
## 实现方法
自定义模块，继承`torch.nn.Module`并定义`forward`函数

In [1]:
import torch

## 产生数据
使用`torch.randn()`随机生成满足标准正态分布的张量，size为$1000\times2$。

In [2]:
x = torch.randn(1000, 2)
x

tensor([[ 6.8009e-01,  2.4196e-02],
        [ 1.1357e+00,  9.0336e-02],
        [ 8.5783e-04, -7.1526e-01],
        ...,
        [ 3.5624e-01,  1.8211e-02],
        [-4.7350e-01,  8.3995e-01],
        [ 3.0120e-01, -2.5702e+00]])

使用`torch.sum(input, dim, keepdim=False, dtype=None)`生成label，其参数如下：
* `input`：需要求和的tensor
* `dim`：需要求和的维度
* `keepdim`：默认为False，如果为True则求和后的输出的维数与input相同

也可以使用`y[x.sum(dim=1) >= 0] = 1`生成label。

In [3]:
y = torch.zeros(1000, 1)
y[torch.sum(x, dim=1) >= 0] = 1
# y[x.sum(dim=1) >= 0] = 1
y

tensor([[1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
      

## 建立模型并训练
### 建立网络
自定义模块，继承`torch.nn.Module`并定义`forward`函数

In [4]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        relu = self.relu(self.linear1(x))
        y_pred = self.sigmoid(self.linear2(x))
        return y_pred

### 损失函数
损失函数使用`torch.nn.BCELoss()`，即Binary Cross Entrophy Loss，适用于使用Sigmoid激活函数的二分类问题。
* `input`: Tensor of arbitrary shape
* `target`: Tensor of the same shape as input

### 优化器
使用`torch.optim.Adam()`进行梯度下降，其重要的参数为：
* `params`：网络的权重，通过`model.parameters()`获取，为可迭代类型
* `lr`：学习率learning_rate，默认为1e-3

In [5]:
model = TwoLayerNet(2, 4, 1)
learning_rate = 1e-3
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

TwoLayerNet(
  (linear1): Linear(in_features=2, out_features=4, bias=True)
  (linear2): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


### 自动求导
* `optimizer.zero_grad()`：在反向传播之前，使用optimizer将它要更新的所有张量的梯度清零(这些张量是模型可学习的权重)
* `loss.backward()`：反向传播：根据模型的参数计算loss的梯度
* `optimizer.step()`：调用Optimizer的step函数使它所有参数更新

也可将以上三条语句替换为以下代码：
```python
# 反向传播之前清零梯度
model.zero_grad()
loss.backward()
# 使用梯度下降更新权重。
# 每个参数都是张量，所以我们可以像我们以前那样可以得到它的数值和梯度
with torch.no_grad():
    for param in model.parameters():
        param -= learning_rate * param.grad
```
此时不使用优化器，而是在`torch.no_grad()`上下文环境中更新梯度。

In [6]:
for t in range(1000):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

RuntimeError: size mismatch, m1: [1000 x 2], m2: [4 x 1] at C:\w\1\s\windows\pytorch\aten\src\TH/generic/THTensorMath.cpp:136

## 创建测试样例并进行测试

In [6]:
x_test = torch.randn(100, 2)
x_test

tensor([[ 0.6483, -1.7291],
        [-0.8005,  0.1327],
        [-0.9405, -1.1010],
        [-0.7488, -1.1779],
        [-2.0758, -0.3546],
        [-1.8351,  1.1915],
        [-1.1949, -0.2665],
        [-0.1990, -0.1263],
        [ 0.5126,  0.6300],
        [-0.3034,  1.6364],
        [-1.3917,  0.2670],
        [-1.5884, -3.0683],
        [ 0.2465,  0.5223],
        [-1.1889, -0.7998],
        [ 1.1082,  0.1497],
        [ 0.7781, -0.1030],
        [-0.3398,  0.5734],
        [-0.3579, -1.9046],
        [-0.7103,  1.5194],
        [ 1.1019, -0.3515],
        [ 1.7460, -1.1121],
        [-0.1986, -0.9290],
        [-0.2504,  0.0566],
        [-1.0126,  0.2362],
        [ 0.5543,  0.5620],
        [-1.2862,  0.2296],
        [ 0.2894,  0.3309],
        [ 1.4080, -0.3574],
        [-0.3311, -1.8519],
        [ 0.9288, -1.0814],
        [-0.6169, -0.0170],
        [-0.8562,  2.5685],
        [-0.1703, -0.9603],
        [ 0.9163, -1.4430],
        [-1.1216,  1.6422],
        [-1.3011,  1

In [7]:
y_test = torch.zeros(100, 1)
y_test[torch.sum(x_test, dim=1) >= 0] = 1
# y_test[x_test.sum(dim=1) >= 0] = 1
y_test

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
      

In [8]:
y_test_pred = torch.zeros(100, 1)
y_test_pred[model(x_test) >= .5] = 1
y_test_pred

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
      

In [9]:
result = torch.zeros(100, 1)
result[y_test == y_test_pred] = 1
print('accurate: {}'.format(result.sum().item() / 100))

accurate: 1.0
